In [1]:
# <수행 방법 요약>
# 이미지에서 각 모델들이 인식한 객체들에게서 바운딩된 박스, 신뢰도, 라벨들을 추출한다.
# 모든 박스들 중에서 신뢰도가 가장 높은 박스를 추출한다.
# 이미지에 추출한 박스의 좌표, 신뢰도, 라벨 이름을 표시한다.

from ultralytics import YOLO
import numpy as np
from torchvision.ops import nms
import torch
import cv2

# 모델 파일명 리스트(밴: model10)
model_files = [f'../models_ensemble/models/model{model_number}.pt' for model_number in [1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 12]]

# 모델 로드 및 names 저장
models = []
model_names = {}

# 모델파일 이름, 모델, 클래스 저장
for model_file in model_files:
    model = YOLO(model_file)
    models.append((model_file, model)) # 모델 파일 이름도 같이 저장함
    model_names[model_file] = model.names

In [2]:
def ensemble_predict(image):
    results = []
    detection_counts = []
    for model in models:
        print(f'\n{model[0]}', end="")
        result = model[1].predict(image, conf=0.5)
        results.append(result)
        # 각 모델에서 감지한 바운딩 박스의 수
        detection_counts.append(len(result[0].boxes))
    print('\ndetection_counts:')
    print(detection_counts)
    print('\n' + '==' * 50)

    combined_results = combine_results(*results) # final_boxes, final_confidences, final_labels
    return combined_results

def combine_results(*results):
    combined_boxes = []
    combined_confidences = []
    combined_labels = []  # 각 박스의 라벨을 저장하기 위한 리스트

    # 각 모델의 결과에서 바운딩 박스를 추출하여 결합
    for model_index, result_list in enumerate(results):
        model_name = models[model_index][0]
        for result in result_list:
            for box in result.boxes:
                x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
                conf = box.conf[0].cpu().numpy()
                class_id = int(box.cls[0].cpu().numpy())
                
                # 클래스 번호를 클래스 이름으로 변환
                class_name = model_names[model_name][class_id]
                
                combined_boxes.append([x1, y1, x2, y2])
                combined_confidences.append(conf)
                combined_labels.append(class_name)

    # NMS (Non-Maximum Suppression) 적용
    boxes = np.array(combined_boxes)
    # print('boxes:')
    # print(boxes)
    confidences = np.array(combined_confidences)

    indices = nms(torch.tensor(boxes), torch.tensor(confidences), 0.4)
    # print('indices:')
    # print(indices)
    final_boxes = boxes[indices]
    final_confidences = confidences[indices]
    
    # NMS 후 최종 박스와 일치하는 라벨 선택
    final_labels = [combined_labels[i] for i in indices]

    return final_boxes, final_confidences, final_labels

# 이미지 파일을 읽어오기
image_path = 'potato.jpg'
image = cv2.imread(image_path)

# 이미지에 대해 앙상블 예측 수행
combined_results = ensemble_predict(image)

# 결과 출력 및 이미지에 그리기
boxes, confidences, labels = combined_results

print("\ndetect:")
for box, conf, label in zip(boxes, confidences, labels):
    x1, y1, x2, y2 = map(int, box)
    label_name = label  # 이미 label은 클래스 이름입니다.

    # 레이블 이름이 확인되었으면, 이미지를 수정합니다.
    if label_name is not None:
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(image, f'{label_name} {conf:.2f}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
        print(f'\nBox: {box}, Confidence: {conf}, Label: {label_name}')

# 수정된 이미지 저장
output_image_path = 'potato_with_boxes.jpg'
cv2.imwrite(output_image_path, image)
print(f"\nAnnotated image saved as {output_image_path}")



../models_ensemble/models/model1.pt
0: 640x640 (no detections), 19.5ms
Speed: 3.0ms preprocess, 19.5ms inference, 42.0ms postprocess per image at shape (1, 3, 640, 640)

../models_ensemble/models/model2.pt
0: 640x640 2 potatos, 18.0ms
Speed: 2.0ms preprocess, 18.0ms inference, 79.4ms postprocess per image at shape (1, 3, 640, 640)

../models_ensemble/models/model3.pt
0: 640x640 2 potatos, 19.0ms
Speed: 2.5ms preprocess, 19.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

../models_ensemble/models/model4.pt
0: 640x640 2 potatos, 21.0ms
Speed: 1.0ms preprocess, 21.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

../models_ensemble/models/model5.pt
0: 640x640 2 Eggs, 20.0ms
Speed: 2.0ms preprocess, 20.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

../models_ensemble/models/model6.pt
0: 640x640 (no detections), 20.0ms
Speed: 2.0ms preprocess, 20.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

../models_

In [8]:
import os

def process_and_predict_images(image_dir, output_dir):
    # 이미지 디렉토리 내 모든 파일 읽기
    for filename in os.listdir(image_dir):
        image_path = os.path.join(image_dir, filename)
        if os.path.isfile(image_path):
            image = cv2.imread(image_path)

            if image is not None:
                # 이미지에 대해 앙상블 예측 수행
                combined_results = ensemble_predict(image)  # final_boxes, final_confidences, final_labels

                # 예측 결과가 올바른 형식인지 확인
                if len(combined_results) != 3 or not all(isinstance(x, (list, np.ndarray)) for x in combined_results):
                    print(f"Unexpected result format from ensemble_predict for {filename}")
                    continue

                # 결과 출력 및 이미지에 그리기
                boxes, confidences, labels = combined_results

                print(f"\nResults for {filename}:")
                for box, conf, label in zip(boxes, confidences, labels):
                    x1, y1, x2, y2 = map(int, box)
                    label_name = label  # 이미 label은 클래스 이름입니다.

                    # 레이블 이름이 확인되었으면, 이미지를 수정합니다.
                    if label_name is not None:
                        # 사각형 테두리 그리는 함수
                        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)

                        # 텍스트 추가하는 함수
                        cv2.putText(image, f'{label_name} {conf:.2f}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

                        # 박스 좌표, 신뢰도, 라벨이름 출력
                        print(f'Box: {box}, Confidence: {conf}, Label: {label_name}')

                # 수정된 이미지 저장
                output_image_path = os.path.join(output_dir, f"predict_{filename}")
                cv2.imwrite(output_image_path, image)
                print(f"Annotated image saved as {output_image_path}")

# 이미지 디렉토리와 출력 디렉토리 지정
image_dir = 'images'
output_dir = 'predict_nms'

# 출력 디렉토리가 존재하지 않으면 생성
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# 이미지 처리 및 예측 수행
process_and_predict_images(image_dir, output_dir)



../models_ensemble/models/model1.pt
0: 384x640 1 carrot, 173.0ms
Speed: 2.3ms preprocess, 173.0ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

../models_ensemble/models/model2.pt
0: 384x640 1 Mushroom, 92.5ms
Speed: 2.2ms preprocess, 92.5ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

../models_ensemble/models/model3.pt
0: 384x640 (no detections), 72.3ms
Speed: 2.2ms preprocess, 72.3ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

../models_ensemble/models/model4.pt
0: 384x640 (no detections), 20.3ms
Speed: 0.7ms preprocess, 20.3ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

../models_ensemble/models/model5.pt
0: 384x640 (no detections), 21.0ms
Speed: 0.7ms preprocess, 21.0ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

../models_ensemble/models/model6.pt
0: 384x640 (no detections), 19.5ms
Speed: 0.8ms preprocess, 19.5ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 6